In [1]:
from eyemind.dataloading.load_dataset import *


/Users/rickgentry/miniforge3/envs/eyemind/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
def sequence_length_map(folder):
    seq_map = {}
    for f in Path(folder).glob("*.csv"):
        df = pd.read_csv(f)
        seq_len = len(df["XAvg"])
        seq_map[f.name] = seq_len
    return seq_map

def filter_by_min_seq(seq_map, min_seq_len=500):
    res = []
    count_filtered=0
    total_seq = 0
    for k, v in seq_map.items():
        if v > min_seq_len:
            res.append(k)
        else:
            count_filtered+=1
        total_seq+=1
    print(count_filtered)
    print(total_seq)
    return res

In [18]:
seq_map = sequence_length_map(Path("/Users/rickgentry/emotive_lab/eyemind/data/processed/fixation"))

In [25]:
filter_by_min_seq(seq_map,1000)

484
6279


['EML1_051-Hypotheses9.csv',
 'EML1_127-Validity8.csv',
 'EML1_075-Hypotheses8.csv',
 'EML1_005-Bias4.csv',
 'EML1_115-Bias6.csv',
 'EML1_072-CausalClaims4.csv',
 'EML1_017-Variables9.csv',
 'EML1_110-CausalClaims0.csv',
 'EML1_042-Bias9.csv',
 'EML1_111-CausalClaims6.csv',
 'EML1_120-Variables5.csv',
 'EML1_135-Variables5.csv',
 'EML1_053-Bias6.csv',
 'EML1_128-Variables4.csv',
 'EML1_104-Bias9.csv',
 'EML1_129-CausalClaims8.csv',
 'EML1_024-Variables6.csv',
 'EML1_038-Validity1.csv',
 'EML1_135-CausalClaims9.csv',
 'EML1_097-Bias5.csv',
 'EML1_031-Variables6.csv',
 'EML1_048-Hypotheses8.csv',
 'EML1_039-Variables7.csv',
 'EML1_060-Bias0.csv',
 'EML1_076-Validity6.csv',
 'EML1_126-Validity6.csv',
 'EML1_011-Validity5.csv',
 'EML1_021-CausalClaims3.csv',
 'EML1_019-Bias8.csv',
 'EML1_041-Variables2.csv',
 'EML1_085-Validity1.csv',
 'EML1_107-Hypotheses8.csv',
 'EML1_123-Hypotheses9.csv',
 'EML1_036-Bias2.csv',
 'EML1_126-Bias0.csv',
 'EML1_132-Validity3.csv',
 'EML1_062-Validity3.csv',

In [3]:
from pytorch_lightning.utilities.cli import LightningCLI
from eyemind.models.transformers import InformerEncoderDecoderModel, InformerEncoderFixationModel, InformerMultiTaskEncoderDecoder
from eyemind.dataloading.informer_data import InformerDataModule
from eyemind.experiments.cli import FoldsLightningCLI
import yaml

In [4]:
config_path = "/Users/rickgentry/emotive_lab/eyemind/experiment_configs/local/multitask_informer_pretraining.yml"
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

In [6]:
from pytorch_lightning import seed_everything


seed_everything(config["seed_everything"], workers=True)

Global seed set to 42


42

In [7]:
datamodule = InformerDataModule(**config["data"])


In [8]:
datamodule.setup()

In [13]:
train_indices = datamodule.train_dataset.indices
train_files, train_labels = datamodule.train_dataset.dataset.get_files_from_indices(train_indices), datamodule.train_dataset.dataset.get_labels_from_indices(train_indices)


In [14]:
len(train_indices), len(train_files)

(4472, 4472)

In [15]:
files = [f.split(".")[0] for f in train_files]


In [53]:
files

['EML1_018-Bias4',
 'EML1_017-CausalClaims1',
 'EML1_016-Hypotheses6',
 'EML1_101-Variables1',
 'EML1_015-Hypotheses8',
 'EML1_097-Validity1',
 'EML1_093-CausalClaims8',
 'EML1_090-Validity8',
 'EML1_107-Variables2',
 'EML1_085-Variables1',
 'EML1_132-Validity4',
 'EML1_086-Bias9',
 'EML1_135-Bias8',
 'EML1_031-Bias0',
 'EML1_115-Bias4',
 'EML1_036-Bias0',
 'EML1_062-Hypotheses1',
 'EML1_081-CausalClaims3',
 'EML1_063-Variables5',
 'EML1_066-CausalClaims1',
 'EML1_064-Hypotheses8',
 'EML1_111-CausalClaims8',
 'EML1_047-Validity1',
 'EML1_085-Variables4',
 'EML1_093-Validity3',
 'EML1_086-Variables1',
 'EML1_008-Hypotheses2',
 'EML1_066-Validity8',
 'EML1_089-Validity4',
 'EML1_055-Variables2',
 'EML1_083-Bias9',
 'EML1_075-Validity0',
 'EML1_092-Variables1',
 'EML1_085-Bias7',
 'EML1_022-Bias3',
 'EML1_047-Hypotheses9',
 'EML1_035-CausalClaims3',
 'EML1_095-Validity2',
 'EML1_072-Bias5',
 'EML1_132-CausalClaims6',
 'EML1_099-Validity3',
 'EML1_062-CausalClaims2',
 'EML1_131-Variables9'

In [16]:
len(files)

4472

In [24]:
len(datamodule.label_df)

7977

In [34]:
label_df = datamodule.label_df
label_df = label_df[label_df["filename"].isin(files)]
label_df = label_df.drop_duplicates("filename")

In [36]:
label_df.to_csv("/Users/rickgentry/emotive_lab/eyemind/data/EML1_pageLevel_with_filename_nodups.csv",)

In [35]:
len(label_df["ParticipantID"].values), len(label_df["filename"].values)

(4472, 4472)

In [28]:
set_labels = set(label_df["filename"].values.tolist())

In [29]:
len(set_labels)

4472

In [31]:
label_df[label_df.duplicated(subset="filename")]

,Unnamed: 0,ParticipantID,Text,PageNum,datetime,unix_start,unix_end,readtime,MW,SVT,...,Deep_X,Rote_Y,Inference_Y,Rote_Z,Inference_Z,Deep_Z,Rote_D,Inference_D,filename,sequence_length
3604,3604,EML1_076,Bias,1,2020-11-23 16:20:45.003,1.606174e+09,1.606174e+09,55.738,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,EML1_076-Bias0,3279


In [33]:
label_df[label_df["filename"] == "EML1_076-Bias0"]

,Unnamed: 0,ParticipantID,Text,PageNum,datetime,unix_start,unix_end,readtime,MW,SVT,...,Deep_X,Rote_Y,Inference_Y,Rote_Z,Inference_Z,Deep_Z,Rote_D,Inference_D,filename,sequence_length
3603,3603,EML1_076,Bias,1,2020-11-23 12:36:31.637,1.606160e+09,1.606160e+09,55.738,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,EML1_076-Bias0,3279
3604,3604,EML1_076,Bias,1,2020-11-23 16:20:45.003,1.606174e+09,1.606174e+09,55.738,NaN,NaN,...,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,EML1_076-Bias0,3279


In [54]:
fold_path = "/Users/rickgentry/emotive_lab/eyemind/data_splits/4fold_participant.yml"
with open(fold_path, "r") as f:
    folds = yaml.safe_load(f)

In [58]:
with open(fold_path, "r") as f:
    folds2 = yaml.safe_load(f)

In [59]:
train2_fold0, val2_fold0 = folds2["folds"][0]['train'], folds2["folds"][0]['val']

In [62]:
strain == set(train2_fold0)

True

In [55]:
train_fold0, val_fold0 = folds["folds"][0]['train'], folds["folds"][0]['val']

In [61]:
strain = set(train_fold0)
sval = set(val_fold0)

In [52]:
strain.intersection(sval)

set()

In [56]:
train_partID_fold0 = set([s.split("-")[0] for s in train_fold0])
val_partID_fold0 = set([s.split("-")[0] for s in val_fold0])


In [57]:
train_partID_fold0.intersection(val_partID_fold0)

set()